# Notebook 5: Dune Migration
## Animating changes in topography over time and computing erosion rates

***

## 1. Import Python Packages

In [1]:
# Import Python standard library and IPython packages we need.
import os
import sys
import subprocess
from IPython.display import Image

# Ask GRASS GIS where its Python packages are.
grass_call = "grass83"
sys.path.append(
    subprocess.check_output([grass_call, "--config", "python_path"], text=True, shell=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj


***

## 2. Start GRASS Session

In [7]:
gj.init("nags_head/PERMANENT")

***

## 3. Compute  DEM of Dirrerence (DoD)

In [11]:
# Purpose: Compute a DEM of Difference (DoD).
r.mapcalc "NH_total_change = NH_2008_1m - NH_1999_1m"
r.colors map=NH_total_change 
rules=color_elevation_diff.txt

SyntaxError: invalid syntax (880463402.py, line 2)

***

## 4. Visualization

In [4]:
#timeseriesmap
timeseriesmap = gj.TimeSeriesMap()
timseriesemap.d_rast(map="elevation_shade")
timeseriesmap.add_raster_series("NH_total_change")
timeseriesmap.d_legend(at=(1, 40, 90, 95))
timeseriesmap.show()
